# 1. Hello, World! 
## 1.1 This is a simple demo
The first block of code is initialization. All subsequent code must be executed after this block has been run.

In [1]:
# pip install semantic-kernel==0.9.6b1

In [2]:
# pip install --upgrade pydantic

In [5]:
import os
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# Load .env into environment variables
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Create semantic kernel
kernel = sk.Kernel()

# Configure OpenAI service. OPENAI_BASE_URL will be automatically loaded and applied
api_key = os.getenv('OPENAI_API_KEY')
service_id = "default"
service = OpenAIChatCompletion(
    service_id=service_id,
    ai_model_id="gpt-3.5-turbo-1106",
    api_key=api_key
)

# Add the LLM service to the kernel
# Multiple services can be added. The first added will be used by default, non-default ones must be specified to use
kernel.add_service(service)


Prompt calls a large model, which is treated as a Semantic Function (detailed below).

In [6]:
import asyncio
# Define semantic function
joke_function = kernel.add_function(
    function_name='joke',  # Function name, required
    plugin_name="MyDemoPlugin",  # Plugin the function belongs to, required
    prompt="Tell a joke"  # Prompt, required
)

# Run the function and check the result
result = await kernel.invoke(joke_function)
print(result)

Why don't skeletons fight each other?

They don't have the guts!


The above code is executed in the form of a Jupyter notebook. If running locally, please refer to the following format.

In [7]:
# import asyncio
# 
# async def run_function(*args):
#     return await kernel.invoke(*args)
# 
# result = asyncio.run(
#     run_function(joke_function)
# )

<div class="alert alert-success">
<b>Key Points:</b>
Using an analogy with the operating system we're familiar with can help better understand SK.
<ol>
<li>Start the operating system: <code>kernel = sk.Kernel()</code></li>
<li>Install drivers: <code>kernel.add_xxx_service()</code></li>
<li>Install applications: <code>func = kernel.create_semantic_function()</code></li>
<li>Run applications: <code>kernel.invoke(func...)</code></li>
</ol>
</div>

<div class="alert white-background" style="background-color: white; color: black; padding: 10px;">
    The following diagram shows the basic architecture of SK.
</div>

## 1.2 Overview 

#### Mind-and-Body-of-Semantic-Kernel:

<img src="data/mind-and-body-of-semantic-kernel.png" style="margin-left: 0px" width=500px>


#### Plugins:
##### Review the design concepts of microservices: Applications use SK to call Plugins to complete various tasks.
<img src="data/plugins.png" style="margin-left: 0px" width=700px>

#### Kernel:

<img src="data/kernel.png" style="margin-left: 0px" width=700px>


# 2. Prompt Template

In [8]:
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.functions import KernelArguments

# Get the default settings
reg_settings = kernel.get_service(service_id).get_prompt_execution_settings_class()(service_id=service_id)

# Define the Prompt template
# In the template, variables are represented as {$topic}
prompt_template_config = PromptTemplateConfig(
    template="Tell a joke about {{$topic}}",
    description="Generate a joke about a specific topic",
    execution_settings={service_id: reg_settings},
    input_variables=[
        InputVariable(name="topic", description="The topic", is_required=True)
    ],
)

# Register the function
topical_joke_function = kernel.add_function(
    function_name="topic_joke",
    plugin_name="MyDemoPlugin",
    prompt_template_config=prompt_template_config,
)

In [9]:
# Run the function and get the result
result = await kernel.invoke(
    topical_joke_function,
    KernelArguments(topic="Eric") # 传入参数
)
print(result)

Why did Eric bring a ladder to the bar?

Because he heard the drinks were on the house!


# 3. Semantic Functions

In [8]:
# Load semantic function. Pay attention to the directory structure
my_plugins = kernel.add_plugin(parent_directory="./demo", plugin_name="MyPlugins")

func = my_plugins["Text2SQL"]

# Execute
result = await kernel.invoke(
    func,
    KernelArguments(input="What courses are available in April 2024"),
)
print(result)



SELECT * FROM Courses WHERE course_date BETWEEN '2024-04-01' AND '2024-04-30';


## 3.1 Multiple Input Variables

In [9]:
prompt = """Conversation history is as follows:
{{ $history }}
---
User: {{ $request }}
Assistant: """
# Define Prompt Template
# In the template, variables are represented as {{ variable_name }}
prompt_template_config = PromptTemplateConfig(
    template=prompt,
    description="Multi-turn dialogue",
    execution_settings={service_id: reg_settings},
    input_variables=[
        InputVariable(name="request", description="The user input", is_required=True),
        InputVariable(name="history", description="The dialogue history", is_required=True),
    ],
)

# Register function
chat = kernel.add_function(
    function_name="chat",
    plugin_name="MyDemoPlugin",
    prompt_template_config=prompt_template_config,
)


In [10]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good at answering users' questions.")

while True:
    request = input('User: ').strip()
    if not request:
        break
    result = await kernel.invoke(
        chat,
        KernelArguments(
        request=request,
        history=chat_history
        ),
    )
    print(f'Assistant: {result}')
    chat_history.add_user_message(request)
    chat_history.add_assistant_message(str(result))

Assistant: Hello, Eric! How can I assist you today?
Assistant: As an AI, I don't have feelings, but I'm here and ready to help you with any questions or concerns you may have. How can I assist you today?


## 3.2 Native Funcitons

In [11]:
from semantic_kernel.functions  import kernel_function


class DBConnectorPlugin:
    def __init__(self, db_cursor):
        self.db_cursor = db_cursor

    @kernel_function(
        description="查询数据库", # function description
        name="query_database", # function name
    )
    def exec(self, sql_exp: str) -> str:
        self.db_cursor.execute(sql_exp)
        records = cursor.fetchall()
        return str(records)

In [12]:
# define native data and SQL DB 

import sqlite3

# creat connection of DB
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# create table
cursor.execute("""
CREATE TABLE Courses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    course_date DATE NOT NULL,
    start_time TIME NOT NULL,
    end_time TIME NOT NULL,
    course_name VARCHAR(255) NOT NULL,
    instructor VARCHAR(255) NOT NULL
);
""")

# Insert 5 specific sample records
timetable = [
    ('2024-01-23', '20:00', '22:00', 'Basics of English Writing', 'John'),
    ('2024-01-25', '20:00', '22:00', 'Advanced English Grammar', 'John'),
    ('2024-01-29', '20:00', '22:00', 'English Literature: An Introduction', 'Mary'),
    ('2024-02-20', '20:00', '22:00', 'Conversational English', 'David'),
    ('2024-02-22', '20:00', '22:00', 'English for Business Communication', 'John'),
    ('2024-02-29', '20:00', '22:00', 'Public Speaking and Presentation Skills', 'Alice'),
    ('2024-03-05', '20:00', '22:00', 'English Phonetics and Phonology', 'Alice'),
    ('2024-03-07', '20:00', '22:00', 'Creative Writing', 'Alice'),
    ('2024-03-14', '20:00', '22:00', 'Preparation for TOEFL', 'Alice'),
    ('2024-03-19', '20:00', '22:00', 'English Vocabulary Building', 'Alice'),
    ('2024-03-21', '20:00', '22:00', 'English for Academic Purposes', 'Alice'),
    ('2024-03-26', '20:00', '22:00', 'Reading Comprehension Techniques (Part 1)', 'Alice'),
    ('2024-03-28', '20:00', '22:00', 'Reading Comprehension Techniques (Part 2)', 'Alice'),
    ('2024-04-09', '20:00', '22:00', 'Understanding English Idioms (Part 1)', 'Mark'),
    ('2024-04-11', '20:00', '22:00', 'Understanding English Idioms (Part 2)', 'Mark'),
    ('2024-04-16', '20:00', '22:00', 'Understanding English Idioms (Part 3)', 'Mark'),
    ('2024-04-18', '20:00', '22:00', 'English for Travel (Part 1)', 'Sophia'),
    ('2024-04-23', '20:00', '22:00', 'English for Travel (Part 2)', 'Sophia'),
    ('2024-04-25', '20:00', '22:00', 'Career Opportunities in English Language', 'John'),
    ('2024-05-07', '20:00', '22:00', 'English for Customer Service', 'John'),
    ('2024-05-09', '20:00', '22:00', 'Introduction to English Linguistics', 'John'),
    ('2024-05-14', '20:00', '22:00', 'English for Project Management', 'Alice'),
]

for record in timetable:
    cursor.execute('''
    INSERT INTO Courses (course_date, start_time, end_time, course_name, instructor)
    VALUES (?, ?, ?, ?, ?)
    ''', record)

# Commit the transaction
conn.commit()

In [13]:
# load native function
db_connector = kernel.add_plugin(DBConnectorPlugin(cursor), "DBConnectorPlugin")

# return the result
result = await kernel.invoke(
    db_connector["query_database"],
    KernelArguments(
        sql_exp="SELECT COUNT(*) as count FROM Courses WHERE instructor = 'Alice'"
    ),
)
print(result)

[(9,)]


another native function method is as following  

In [14]:
result = await db_connector["query_database"](
    kernel, sql_exp="SELECT COUNT(*) as count FROM Courses WHERE instructor = 'Alice'"
)
print(result)

[(9,)]


## 3.3 Function Parameter Annotation


In [15]:
from typing import Annotated

class DBConnector:
    def __init__(self, db_cursor):
        self.db_cursor = db_cursor

    @kernel_function(
        description="数据库查询",  # function 描述
        name="query_database"  # function 名字
    )
    def query(
        self,
        sql_exp: Annotated[str, "SQL查询表达式"]
    ) -> Annotated[str, "数据库查询结果"]:
        records = self.db_cursor.execute(sql_exp)
        return records.fetchall()

## 3.4 Predefined functions in Semantic Kernel

The method to load them:

```python
from semantic_kernel.core_plugins import <PluginName>
```

These plugins are:

- [`ConversationSummaryPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/conversation_summary_plugin.py) - Generates a summary of a conversation.
- [`HttpPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/http_plugin.py) - Sends HTTP requests, supporting GET, POST, PUT, and DELETE methods.
- [`MathPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/math_plugin.py) - Performs addition and subtraction operations.
- [`TextMemoryPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/text_memory_plugin.py) - Saves text into memory, enabling vector search functionality.
- [`TextPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/text_plugin.py) - Converts text to uppercase or lowercase and trims leading/trailing spaces.
- [`TimePlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/time_plugin.py) - Retrieves the current time and formats time parameters in various ways.
- [`WaitPlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/wait_plugin.py) - Waits for a specified amount of time.
- [`WebSearchEnginePlugin`](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/core_plugins/web_search_engine_plugin.py) - Searches the internet for the provided text.

--- 

This list shows built-in plugins in SK that can help in different tasks, from HTTP requests to text transformations and time management. Each plugin can be easily imported and used within the Semantic Kernel environment.

# 4. Nested function calls

## 4.1 Nested function calls of Semantic Function 

In [16]:
translate_prompt = """
将中文词'{{$chinese}}'翻译为日语
直接给出一个翻译结果，不要评论。
尽可能用Hanji表示。
"""

In [17]:
joke_prompt = """
'{{$input}}'的日语是：{{MyDemoPlugin.translate $input}}
根据以上词汇在中日文中的语义差异，讲一个笑话。
"""

In [18]:
import os
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# 加载 .env 到环境变量
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())

# 创建 semantic kernel
kernel = sk.Kernel()

# 配置OpenAi 服务。 OPENAI_BASE_URL 会被自动加载生效
api_key = os.getenv('OPENAI_API_KEY')
service_id = "default"

# 将 LLM 服务添加到 kernel 中
kernel.add_service(
    OpenAIChatCompletion(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        api_key=api_key
    ),
)

In [19]:
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.functions import KernelArguments

req_settings = kernel.get_service(service_id).get_prompt_execution_settings_class()(service_id=service_id)

trans_prompt_template_config = PromptTemplateConfig(
    template=translate_prompt,
    description="Translate Chinese to Japanese",
    execution_settings={service_id:req_settings},
    input_variables=[
        InputVariable(name="chinese", description="The source", is_required=True)
    ]
)

joke_prompt_template_config = PromptTemplateConfig(
    template=joke_prompt,
    description="Generate a joke about a specific topic",
    execution_settings={service_id:req_settings},
    input_variables=[
        InputVariable(name="input", description="The topic", is_required=True)
    ]
)

translate_function = kernel.add_function(
    function_name="translate",
    plugin_name="MyDemoPlugin",
    prompt_template_config=trans_prompt_template_config,
)

joke_function = kernel.add_function(
    function_name="joke",
    plugin_name="MyDemoPlugin",
    prompt_template_config=joke_prompt_template_config,
)

In [20]:
result = await kernel.invoke(
    joke_function,
    KernelArguments(
        input="信件"
    )
)

print(result)

为了给日本朋友寄信，我学了很多日语词汇。但是当我寄出了一封“手紙”后，他却收到了一封真正的手纸！看来我还需要再多学习一下日语的语义差异啊！


## 4.2 Prompt template call Native Function

In [21]:
prompt = """
已知，数据库形式为
CREATE TABLE Courses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    course_date DATE NOT NULL,
    start_time TIME NOT NULL,
    end_time TIME NOT NULL,
    course_name VARCHAR(255) NOT NULL,
    instructor VARCHAR(255) NOT NULL
);

用自然语言解释用户的SQL查询的意图和查询结果

用户输入：{{$input}}

查询结果：{{DBConnectorPlugin.query_database $input}}
"""


In [22]:
# load native function
kernel.add_plugin(DBConnectorPlugin(cursor), "DBConnectorPlugin")

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    description="查询数据库",
    execution_setting={service_id:reg_settings},
    input_variable=[
        InputVariable(name="input",decription="The user query", is_required=True),
    ],
)

db_query_function = kernel.add_function(
    function_name="db_query",
    plugin_name="MyDemoPlugin",
    prompt_template_config=prompt_template_config,
)

In [23]:
result = await kernel.invoke(
    db_query_function,
    KernelArguments(
        input="SELECT COUNT(*) as count FROM Courses WHERE instructor = 'Eric'"
    )
)

print(result)

用户的SQL查询意图是要统计出所有由Eric教授的课程的数量。查询结果显示没有由Eric教授的课程，因为返回的结果是0。


# 5. Memory
Using the memory in SK is very simple:
1. Use kernel.add_service() to add a large text vector generation service.
2. Use kernel.add_plugin() to add a connection to a vector database.
3. Use memory.save_information() to save information to the memory store.
4. Use memory.search() to search for information.

Using the README.md data from ChatALL, using memory as the memory store, we do not base conversations on documents.

## 5.1 Initiate Embedding

In [24]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
import os

# Load .env into environment variables
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Create the semantic kernel
kernel = sk.Kernel()

# Configure the OpenAI service. OPENAI_BASE_URL will be automatically loaded and take effect
api_key = os.getenv('OPENAI_API_KEY')
service_id = "default"

llm_service =  OpenAIChatCompletion(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        api_key=api_key
    )

# Add the LLM service to the kernel
kernel.add_service(llm_service)

embedding_gen = OpenAITextEmbedding(
    ai_model_id="text-embedding-ada-002",
    api_key=api_key
)

# Add the Embedding service to the kernel
kernel.add_service(embedding_gen)


In [25]:
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore

# Create a (memory) vector database
memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)

# Add a plugin that connects to the vector database
kernel.add_plugin(TextMemoryPlugin(memory), "TestMemoryPlugin")

KernelPlugin(name='TestMemoryPlugin', description=None, functions={'recall': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='recall', plugin_name='TestMemoryPlugin', description='Recall a fact from the long term memory', parameters=[KernelParameterMetadata(name='ask', description='The information to retrieve', default_value=None, type_='str', is_required=True, type_object=<class 'str'>), KernelParameterMetadata(name='collection', description='The collection to search for information.', default_value='generic', type_='str', is_required=False, type_object=<class 'str'>), KernelParameterMetadata(name='relevance', description='The relevance score, from 0.0 to 1.0; 1.0 means perfect match', default_value=0.75, type_='float', is_required=False, type_object=<class 'float'>), KernelParameterMetadata(name='limit', description='The maximum number of relevant memories to recall.', default_value=1, type_='int', is_required=False, type_object=<class 'int'>)], is_prompt=False, is_asyn

## 5.2  Text Vectorization

In [26]:
from semantic_kernel.text import split_markdown_lines

# Read the file content
with open('data/ChatALL.md', 'r') as f:
    # with open('sk_samples/SamplePlugin/SamplePlugin.py', 'r') as f:
    content = f.read()

# Split the file content into chunks, with a maximum of 100 tokens per chunk (Note: SK’s text split functionality currently supports Chinese less well compared to English)
lines = split_markdown_lines(content, 100)

collection_id = "generic"

# Store the chunked content into memory
for index, line in enumerate(lines):
    await memory.save_information(collection_id, id=index, text=line)

## 5.3 Vector Search

In [27]:
result = await memory.search(collection_id, "ChatALL 怎么下载")
print(result[0].text)

拥有可以访问这些 AI 的帐号，或 API token。
2. 与 AI 网站有可靠的网络连接。

## 下载 / 安装

从 https://github.com/sunner/ChatALL/releases 下载

### Windows 系统

直接下载 \*-win.exe 安装文件并运行之。

### macOS 系统

对于苹果硅芯片 Mac（M1，M2 CPU），请下载 \*-mac-arm64.


## 5.4 Use nested functions to build a simple RAG.

In [28]:
# Create a semantic function directly in the code. This is not recommended in real projects.
# It calls `recall()` inside.
prompt = """
基于下面的背景信息回答问题。如果背景信息为空，或者和问题不相关，请回答"我不知道"。

[背景信息开始]
{{recall $input}}
[背景信息结束]

问题：{{$input}}
回答：
"""


reg_settings = kernel.get_service(service_id).get_prompt_execution_settings_class()(service_id=service_id)

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    description="RAG回答",
    execution_settings={service_id:reg_settings},
    input_variables=[
        InputVariable(name="input", description="The user query", is_requied=True),
    ],
)

reg_function = kernel.add_function(
    function_name="search_and_answer",
    plugin_name="MyDemoPlugin",
    prompt_template_config=prompt_template_config,
)

result = await kernel.invoke(
    reg_function,
    KernelArguments(input="ChatALL 怎么下载")
)

print(result)

从 https://github.com/sunner/ChatALL/releases 下载安装文件。对于Windows系统，下载\*-win.exe文件并运行；对于macOS系统，苹果硅芯片Mac（M1，M2 CPU）下载\*-mac-arm64文件。


## 5.5. Connecting to Other VectorDBs
Semantic Kernel is currently adapted to many mainstream vector databases.
For details, refer to:  https://learn.microsoft.com/en-us/semantic-kernel/memories/vector-db


# 6. Planner

## 6.1 Agent

#### Agent-Overview:

<img src="data/agent-overview.png" style="margin-left: 0px" width=700px>

#### The Differences Between an Agent, RAG, and Copilot : 

<img src="data/agent_2.png" style="margin-left: 0px" width=700px>

## 6.2. SK Python provides four types of Planners:

**SequentialPlanner**  
Creates a plan consisting of a series of steps, where each step is interconnected through custom-generated input and output variables.  
Core: [https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planners/sequential_planner/Plugins/SequentialPlanning/skprompt.txt](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planners/sequential_planner/Plugins/SequentialPlanning/skprompt.txt)  
Official Example: [https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/sequential_planner.py](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/sequential_planner.py)

**ActionPlanner**  
Similar to OpenAI Function Calling, it finds a function to execute from all the registered plugins in the kernel.  
Core prompt: [https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planners/action_planner/skprompt.txt](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planners/action_planner/skprompt.txt)  
Official Example: [https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/action_planner.py](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/action_planner.py)

**StepwisePlanner**  
After executing each step, it performs a review.  
It only outputs actions without executing them.  
Core prompt: [https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planners/stepwise_planner/Plugins/StepwiseStep/skprompt.txt](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planners/stepwise_planner/Plugins/StepwiseStep/skprompt.txt)

**BasicPlanner**  
Not recommended for use. It breaks tasks down and automatically calls various functions to complete them. It is mainly for basic validation and will eventually be replaced by SequentialPlanner.  
Core prompt: [https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/basic_planner.py#L27-L123](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/basic_planner.py#L27-L123)

The steps to use a planner are very simple:

1. Register the plugin to the kernel.
2. Instantiate a planner using the kernel as a parameter.
3. Call the planner's `create_plan_async()` method to obtain a plan.
4. Call the plan's `invoke_async()` method to execute the plan.  
(Note: The interfaces of different planners are not consistent, so they cannot be used interchangeably.)

## 6.3 Implement an Agent
To implement an Agent that can use both search and calendar tools through a Planner, follow these steps:

In [37]:
from semantic_kernel.core_plugins import WebSearchEnginePlugin
from semantic_kernel.connectors.search_engine import  BingConnector
from semantic_kernel.planners import SequentialPlanner
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
import os

# Load .env into environment variables
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Create the semantic kernel
kernel = sk.Kernel()

# Configure the OpenAI service. OPENAI_BASE_URL will be automatically loaded and take effect
api_key = os.getenv('OPENAI_API_KEY')
service_id = "default"

llm_service =  OpenAIChatCompletion(
        service_id=service_id,
        # ai_model_id="gpt-3.5-turbo-1106",
        ai_model_id="gpt-4",
        api_key=api_key
    )

# Add the LLM service to the kernel
kernel.add_service(llm_service)
 

how to register BING API KEY ：[BING API KEY](https://portal.azure.com/#view/Microsoft_Azure_Marketplace/MarketplaceOffersBlade/selectedMenuItemId/home)  --- search with " Bing Custom Search" --- Create a Bing custom search resource and get API key

In [38]:
# import search plugin
connector = BingConnector(api_key=os.getenv("BING_API_KEY"))
kernel.add_plugin(WebSearchEnginePlugin(connector),"WebSearch")

KernelPlugin(name='WebSearch', description=None, functions={'search': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='search', plugin_name='WebSearch', description='Performs a web search for a given query', parameters=[KernelParameterMetadata(name='query', description='The search query', default_value=None, type_='str', is_required=True, type_object=<class 'str'>), KernelParameterMetadata(name='num_results', description='The number of search results to return', default_value=1, type_='int', is_required=False, type_object=<class 'int'>), KernelParameterMetadata(name='offset', description='The number of search results to skip', default_value=0, type_='int', is_required=False, type_object=<class 'int'>)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=None)), method=<bound method WebSearchEnginePlugin.search of <semantic_kernel.core_plugins.web_sear

In [40]:
from semantic_kernel.core_plugins import MathPlugin, TextPlugin, TimePlugin

kernel.add_plugin(TimePlugin(),"time")

# create planner
planner = SequentialPlanner(kernel, service_id)

# begin
# query = "On which date was OpenAI o1 released?"
query = """On which date was OpenAI o1 released?"""

plan = await planner.create_plan(goal=query)
result = await plan.invoke(kernel)

for i,step in enumerate(plan._steps):
    print(step.description)
    print(step.plugin_name + "." + step.name, end=": ")
    print(step.parameters)
    print(step._outputs[0] + "=" + str(result.metadata["results"][i].value))

print(f"Agent 回复: {result}")


# 7. VS Code Plugin

This is a VS Code plugin that allows you to directly create and debug Semantic Functions within VS Code.

Installation link: [https://marketplace.visualstudio.com/items?itemName=ms-semantic-kernel.semantic-kernel](https://marketplace.visualstudio.com/items?itemName=ms-semantic-kernel.semantic-kernel)

# Summary

1. Should I use a development framework?
2. When should I choose SK?

- If you frequently need to switch between different LLMs or have extensive prompt debugging needs, choosing a development framework will make your life easier.
- If your prompt contains a lot of nested calls.
- If you must use the C#/JAVA technology stack, SK might be your only option for now.
- If you're using the Python technology stack, you might want to compare it with LangChain before making a decision.